<a href="https://colab.research.google.com/github/harshjoshei/051_HarshJoshi/blob/main/Lab_6/Assignment/Mnist_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing libraries

from __future__ import absolute_import, division, print_function
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

import numpy as np 
import pandas as pd 
import io
import matplotlib.pyplot as plt

MNIST data set. 
It is a collection of hand-written digits that contains 
60,000 examples - training & 10,000 examples for testing. 
images are (28x28 pixels) with values from 0 to 255. 

Steps followed for each image:

1) converted it to float32

2) normalize to [0, 1]

3) flatten to a 1-D array of 784 features (28*28).

In [3]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32.
x_train = np.array(x_train, np.float32)
x_test = np.array(x_test, np.float32)


# Flatten images to 1-D vector of 784 features (28*28).
num_features=784
x_train = x_train.reshape(60000, num_features)
x_test = x_test.reshape(10000, num_features)


# Normalize images value from [0, 255] to [0, 1].
x_train = x_train / 255
x_test = x_test /255

x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [4]:
x_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [5]:
x_train.shape

(60000, 784)

In [6]:
x_test.shape

(10000, 784)

In [7]:
y_train.shape

(60000,)

In [8]:
y_train.shape

(60000,)

#Step 3: Setting Up Hyperparameters and Data Set Parameters

Initialize the model parameters. 

num_classes denotes the number of outputs, which is 10, as we have digits from 0 to 9 in the data set. 

num_features defines the number of input parameters, and we store 784 since each image contains 784 pixels.

In [9]:
# MNIST dataset parameters.

num_classes = 10 # 0 to 9 digits

num_features = 784 # 28*28

# Training parameters.

learning_rate = 0.01

training_steps = 1000

batch_size = 256

display_step = 50

#Step 4: Shuffling and Batching the Data

We need to shuffle and batch the data before we start the actual training to avoid the model from getting biased by the data. This will allow our data to be more random and helps our model to gain higher accuracies with the test data.

With the help of tf.data.Dataset.from_tensor_slices, we can get the slices of an array in the form of objects. 

The function shuffle(5000) randomizes the order of the data set’s examples. 

Here, 5000 denotes the variable shuffle_buffer, which tells the model to pick a sample randomly from 1 to 5000 samples. 

After that, there are only 4999 samples left in the buffer, so the sample 5001 gets added to the buffer. 

In [10]:
# Use tf.data API to shuffle and batch data.
num_batches = int(x_train.shape[0] / batch_size)
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

#Step 5: Initializing Weights and Biases

We now initialize the weights vector and bias vector with ones and zeros.

In [11]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.

W = tf.Variable(np.random.randn(784, 10).astype(np.float32))

# Bias of shape [10], the total number of classes.
B = tf.Variable(np.random.randn(10).astype(np.float32))

#Step 6: Defining Logistic Regression and Cost Function

We define the logistic_regression function below, which converts the inputs into a probability distribution proportional to the exponents of the inputs using the softmax function. The softmax function, which is implemented using the function tf.nn.softmax, also makes sure that the sum of all the inputs equals one.

In [12]:
# Logistic regression (Wx + b).

def logistic_regression(x):

    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.add(tf.matmul(x, W), B))
    

# Cross-Entropy loss function.

def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth = num_classes)
    

    # Clip prediction values to avoid log(0) error.

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)    

    # Compute cross-entropy.
    loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
    return loss

#Step 7: Defining Optimizers and Accuracy Metrics
When we compute the output, it gives us the probability of the given data to fit a particular class of output. 

We consider the correct prediction as to the class having the highest probability. 

We compute this using the function tf.argmax. 

We also define the stochastic gradient descent as the optimizer from several optimizers present in TensorFlow. We do this using the function tf.optimizers.SGD. 

This function takes in the learning rate as its input, which defines how fast the model should reach its minimum loss or gain the highest accuracy.

In [13]:
# Accuracy metric.

def accuracy(y_pred, y_true):

  # Predicted class is the index of the highest score in prediction vector (i.e. argmax).

  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Step 8: Optimization Process and Updating Weights and Biases
Now we define the run_optimization() method where we update the weights of our model. We calculate the predictions using the logistic_regression(x) method by taking the inputs and find out the loss generated by comparing the predicted value and the original value present in the data set. Next, we compute the gradients using and update the weights of the model with our stochastic gradient descent optimizer.

In [14]:
# Optimization process. 

optimizer = tf.optimizers.SGD(learning_rate)

def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.

    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

    # Compute gradients.

    gradients = g.gradient(loss, [W, B])
    # Stochastic gradient descent optimizer.
    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [W, B]))

In [15]:
# Run training for the given number of steps.

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)
    if step % display_step == 0:

        #Obtain Predictions
        pred = logistic_regression(batch_x)
        #Ccompute loss
        loss = cross_entropy(pred, batch_y)
        #Compute Accuracy
        acc = accuracy(pred, batch_y)
        #print accuracy
        print(f"step: {step}, loss: {loss}, accuracy: {acc}")

step: 50, loss: 252.91900634765625, accuracy: 0.80859375
step: 100, loss: 138.58941650390625, accuracy: 0.890625
step: 150, loss: 169.06842041015625, accuracy: 0.85546875
step: 200, loss: 196.55413818359375, accuracy: 0.87890625
step: 250, loss: 174.19058227539062, accuracy: 0.8828125
step: 300, loss: 87.88935852050781, accuracy: 0.92578125
step: 350, loss: 81.8067855834961, accuracy: 0.890625
step: 400, loss: 120.08135986328125, accuracy: 0.89453125
step: 450, loss: 97.6026611328125, accuracy: 0.8828125
step: 500, loss: 143.742431640625, accuracy: 0.87890625
step: 550, loss: 75.17078399658203, accuracy: 0.8984375
step: 600, loss: 125.49381256103516, accuracy: 0.9140625
step: 650, loss: 114.32486724853516, accuracy: 0.89453125
step: 700, loss: 63.29550552368164, accuracy: 0.9140625
step: 750, loss: 156.4600067138672, accuracy: 0.8359375
step: 800, loss: 113.22059631347656, accuracy: 0.890625
step: 850, loss: 103.51924133300781, accuracy: 0.90234375
step: 900, loss: 119.33432006835938, 

In [16]:
# Test model on validation set.
pred = logistic_regression(x_test)
a = accuracy(pred, y_test)
print(f"Test Accuracy: {a}")

Test Accuracy: 0.8931000232696533
